In [1]:
import typing as T  # pylint: disable=unused-import

import numpy as np
import numpy.typing as npt

from pydrake.solvers import (  # pylint: disable=import-error, no-name-in-module, unused-import
    MathematicalProgram,
    MathematicalProgramResult,
    Solve,
    MosekSolver,
    MosekSolverDetails,
    SolverOptions,
    CommonSolverOption,
)
from pydrake.geometry.optimization import (  # pylint: disable=import-error, no-name-in-module
    HPolyhedron,
    Point,
    ConvexSet,
)

from pydrake.symbolic import ( # pylint: disable=import-error, no-name-in-module, unused-import
    Polynomial,
    Variable,
    Variables,
    Expression,
)  
from program_options import ProgramOptions, FREE_POLY, PSD_POLY

from util import (
    timeit,
    diditwork,
    INFO,
    YAY,
    WARN,
    ERROR,
)  # pylint: disable=import-error, no-name-in-module, unused-import

import plotly.graph_objects as go  # pylint: disable=import-error
from plotly.express.colors import sample_colorscale  # pylint: disable=import-error
import plotly.graph_objs as go
from plotly.subplots import make_subplots

from polynomial_gcs_dual import PolynomialDualGCS, Vertex, Edge
from kink_example import random_uniform_graph_generator, build_m_step_horizon_from_layers

In [9]:
options = ProgramOptions()
options.pot_type = PSD_POLY
options.mosek_tolerance_gap = 1e-9
options.mosek_primal_feas_gap = 1e-9
options.mosek_dual_feas_gap = 1e-9
gcs, layers = random_uniform_graph_generator(options=options)

Solve took 0.208s
solve successful!
-40.254299951370065
SolutionResult.kSolutionFound
Solver is Mosek
time 0.18927502632141113
rescode 0
solution_status 1


In [10]:
v_0 = layers[0][0]
x1,y1,_ = build_m_step_horizon_from_layers(gcs, layers, 1, v_0, 0)
x2,y2,_ = build_m_step_horizon_from_layers(gcs, layers, 2, v_0, 0)
x3,y3,_ = build_m_step_horizon_from_layers(gcs, layers, 3, v_0, 0)

In [11]:
start_vertex = layers[0][0]

fig = gcs.make_plots(start_vertex.name, 0.1)

In [12]:
fig.add_trace(go.Scatter(x=x1, y=y1, mode='lines', name="1-step-lookahead", line=dict(color="red") ))
fig.add_trace(go.Scatter(x=x2, y=y2, mode='lines', name="2-step-lookahead", line=dict(color="purple") ))
fig.add_trace(go.Scatter(x=x3, y=y3, mode='lines', name="3-step-lookahead", line=dict(color="magenta") ))

In [2]:
options = ProgramOptions()
options.use_convex_relaxation = False

graph = PolynomialDualGCS(options)
# test out on something simpel

quad_cost = lambda x, y: np.sum([(x[i]-y[i])**2 for i in range(len(x))])

a1 = graph.AddTargetVertex("a1", Point([0]), lambda x: 0 )
b1 = graph.AddVertex("b1", HPolyhedron.MakeBox([-3],[-1]))
b2 = graph.AddVertex("b2", HPolyhedron.MakeBox([1],[3]))
c1 = graph.AddVertex("c1", HPolyhedron.MakeBox([-3],[3]))

graph.AddEdge(c1, b1, quad_cost)
graph.AddEdge(c1, b2, quad_cost)
graph.AddEdge(b1, a1, quad_cost)
graph.AddEdge(b2, a1, quad_cost)

for v in graph.gcs.Edges():
    print(v.name())

graph.solve_for_true_shortest_path("c1", [2])
graph.solve_for_true_shortest_path("c1", [3])


e0
c1 b1
c1 b2
b1 a1
b2 a1


TypeError: PolynomialDualGCS.solve_for_true_shortest_path() missing 1 required positional argument: 'point'

: 

In [7]:
for v in graph.gcs.Edges():
    print(v.name())

e5
c1 b1
c1 b2
b1 a1
b2 a1
e5
e6
e7
e8
